### Setting up the three databases
With using the cassandra database, this file will create a cassandra node with 3 keyspaces. These keyspaces are:


1. **fish_data**

which is data from Barentwatch API

2. **weather_data**
 
with data from Frost API

Here they are only created, data will be added in the following notebooks:

1. 
2. 



#### Connecting to the cassandra database

In [2]:
# Connecting to Cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [3]:
session.execute(
    "CREATE KEYSPACE IF NOT EXISTS fish_data\
    WITH REPLICATION = {\
        'class' : 'SimpleStrategy',\
        'replication_factor' : 1\
    };"
)

In [4]:
session.execute(
    "CREATE KEYSPACE IF NOT EXISTS weather_data\
    WITH REPLICATION = {\
        'class' : 'SimpleStrategy',\
        'replication_factor' : 1\
    };"
)

In [9]:
session.set_keyspace('fish_data')

table_creation_query = """
    CREATE TABLE IF NOT EXISTS id_35297 (
        datetime timestamp PRIMARY KEY,
        avgAdultFemaleLice FLOAT,
        hasReportedLice BOOLEAN,
        avgMobileLice FLOAT,
        avgStationaryLice FLOAT,
        seaTemperature FLOAT,
    )
    """

session.execute(table_creation_query)